This notebook is designed to find ICESat-2 ground tracks over SnowEx field sites. It is currently configured to be used alongside the 2022/2023 campaign in Alaska, where ICESat-2 TOO orders have been processed to improve coverage of sites near Fairbanks, AK and the Alaskan North Coast.

Finding and downloading the data over the field sites requires icepyx (https://github.com/icesat2py/icepyx). Data can be found for most ICESat-2 products, including the Quick Look products.

In [12]:
import geoviews as gv
import holoviews as hv
import icepyx as ipx
import matplotlib.pyplot as plt
import numpy as np
from osgeo import gdal
import pickle
import rasterio as rio

Setting up a Query object with icepyx requires the shortname for the desired ICESat-2 product, a spatial extent given as a bounding box or shapefile, and a date range. A specific ICESat-2 RGT or list of RGTs may also be requested.

The below query template requests for ICESat-2 data that passes through Fairbanks or the Alaskan Northern coast between September and December 2022. We are looking for ATL08 Quick Look data (ATL08QL), which is preliminary ATL08 data released three days after its acquisition. Two possible queries are given: one that does not specify a RGT (i.e., we will collect all ICESat-2 passes in the region) and one that only seeks a single ICESat-2 overpass.

In [4]:
# Specify the ICESat-2 product (ATL03, ATL06, etc.)
short_name = 'ATL08'

# Define the spatial extent for the queried data. Here, we are loading a bounding box saved as a pickle file.
with open('snowex_sites_for_icepyx.pkl', 'rb') as f:
    coordinates = pickle.load(f)
    spatial_extent = coordinates['alaska']
    
# Define the date range
date_range = ['2022-09-01', '2022-12-31']

# Define the query object
region = ipx.Query(short_name, spatial_extent, date_range) # Request all IS-2 tracks
#region = ipx.Query(short_name, spatial_extent, date_range, tracks='1356') # Request only for a single track

print(region)

Product ATL08 v005
('bounding_box', [-149.65, 64.587, -147.233, 70.202])
Date range ['2022-09-01', '2022-12-31']


Now that we have a Query object, let's see which RGTs may pass through our ROIs in Alaska.

In [6]:
ids = region.avail_granules(ids=True)

# List only the unique RGTs available
rgts = [c[21:25] for c in ids[0]]
print(np.unique(rgts))

['0030' '0083' '0091' '0144' '0152' '0205' '0213' '0266' '0274' '0327'
 '1089' '1097' '1150' '1158' '1272' '1295' '1333' '1356']


In [17]:
# Use the icepyx visualization tool to see our data coverage
cyclemap, rgtmap = region.visualize_elevation()
cyclemap

Generating urls
Sending request to OpenAltimetry, please wait...


100%|██████████| 19/19 [00:05<00:00,  3.30it/s]


Plot elevation, please wait...


:DynamicMap   [cycle]
   :Overlay
      .Tiles.I :Tiles   [x,y]
      .Image.I :Image   [x,y]   (x_y elevation)

Now that we know where the data is, let's see the full list of available data granules.

In [18]:
region.avail_granules(ids=True)

[['ATL08_20220901052025_10891603_005_01.h5',
  'ATL08_20220901180518_10971605_005_01.h5',
  'ATL08_20220905051207_11501603_005_01.h5',
  'ATL08_20220905175700_11581605_005_01.h5',
  'ATL08_20220913045525_12721603_005_01.h5',
  'ATL08_20220914171439_12951605_005_01.h5',
  'ATL08_20220917044704_13331603_005_01.h5',
  'ATL08_20220918170619_13561605_005_01.h5',
  'ATL08_20220922165800_00301705_005_01.h5',
  'ATL08_20220926040446_00831703_005_01.h5',
  'ATL08_20220926164938_00911705_005_01.h5',
  'ATL08_20220930035627_01441703_005_01.h5',
  'ATL08_20220930164120_01521705_005_01.h5',
  'ATL08_20221004034803_02051703_005_01.h5',
  'ATL08_20221004163256_02131705_005_01.h5',
  'ATL08_20221008033943_02661703_005_01.h5',
  'ATL08_20221008162437_02741705_005_01.h5',
  'ATL08_20221012033125_03271703_005_01.h5']]

Let's go ahead and download all of these files!

In [19]:
# NASA Earthdata credentials
uid = 'zhfair'
email = 'zhfair@umich.edu'
region.earthdata_login(uid, email)

Earthdata Login password:  ········


In [20]:
region.order_granules()

Total number of data order requests is  1  for  18  granules.
Data request  1  of  1  is submitting to NSIDC
order ID:  5000003980694
Initial status of your order request at NSIDC is:  processing
Your order status is still  processing  at NSIDC. Please continue waiting... this may take a few moments.
Your order status is still  processing  at NSIDC. Please continue waiting... this may take a few moments.
Your order is: complete
NSIDC returned these messages
['Granule 256879861 contained no data within the spatial and/or temporal '
 'subset constraints to be processed',
 'Granule 256874030 contained no data within the spatial and/or temporal '
 'subset constraints to be processed',
 'Granule 256876459 contained no data within the spatial and/or temporal '
 'subset constraints to be processed']


In [22]:
path = 'ICESat-2/' # Replace with desired folder
region.download_granules(path)

Beginning download of zipped output...
Data request 5000003980694 of  1  order(s) is downloaded.
Download complete


Now that we have downloaded the data, let's get a deeper look at one of the files. We are going to import a few more packages to make this analysis easier.

In [28]:
import h5py
import lidar_processing as lp
import pandas as pd
import sys

In [35]:
# Choose a file
ipx_file = path + 'processed_ATL08_20220918170619_13561605_005_01.h5'
product = ipx_file[19:24]
print(product)

# Load the file of interest
with h5py.File(ipx_file, 'r') as f:
    # Identify the strong beams
    sc_orient = f['orbit_info/sc_orient'][0]
    strong_beams, strong_ids = lp.strong_beam_finder(sc_orient)
    
    # Concatenate the strong beam data into a single DataFrame
    is2_pd = lp.beam_cycle_concat([ipx_file], product)
    
is2_pd.head()

ATL08


,lat,lon,height,delta_time,gt,time
0,65.658195,-147.261246,694.591492,1.487562e+08,5,2022-09-18 17:10:07.737650144
1,65.657303,-147.261490,675.270996,1.487562e+08,5,2022-09-18 17:10:07.751820000
2,65.656410,-147.261734,685.864197,1.487562e+08,5,2022-09-18 17:10:07.765890752
3,65.655518,-147.261993,752.350586,1.487562e+08,5,2022-09-18 17:10:07.779798880
4,65.654625,-147.262238,803.026367,1.487562e+08,5,2022-09-18 17:10:07.793790400
